<h1>This is the last assignment of Capstone Project - The Battle of Neighborhoods</h1>

In [1]:
import pandas as pd
import numpy as np
import requests, json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import datetime
import folium

#Using the geopy to get the coordinates from my street.
address = 'Rua Antenor Bettarello'
geolocator = Nominatim(user_agent="photo_app",timeout=10)
location = geolocator.geocode(address)

#Fousquare use epoch in date time, so this code below will adjust the date (Day-month-year)
def epoch_date(ts_epoch):
      return datetime.datetime.fromtimestamp(ts_epoch).strftime('%d-%m-%Y')

#This is the parameters to use the Foursquare API regular calls.
#### The client_secret was removed purposefully, due to sensitive data####

url = 'https://api.foursquare.com/v2/venues/explore'
params = dict(
client_id='STPRRABCOBMCYXHUP223JL1LJWXV0K1ZW3GLG3HX3FT3YX4Z',
client_secret='SECRET REMOVED',
v='20200404',
ll = '{},{}'.format(location.latitude,location.longitude),
categoryId = '4d4b7105d754a06374d81259', 
limit=500
)

#These lines will get the JSON, returned from call, and create a Data Frame
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
results = data['response']['groups'][0]['items']
dataframe = json_normalize(results)


In [ ]:
#Creating a new dataframe to receive the data from Foursquare Premium calls
columnName = 'venue.location.address', 'venue.name', 'venue.location.lat', 'venue.location.lng', 'venue.categories', 'venue.stats.tipCount', 'venue.price.tier', 'venue.price.message', 'venue.likes.count', 'venue.rating', 'venue.photos.count', 'venue.createdAt', 'venue.shortUrl'
dataframeFull = pd.DataFrame(columns = columnName)

#The Token was removed, due to sensitive Data
oauth_token= "TOKEN Removed"
v='20200404'

#For each venue.ID collected from Regular Call, this part will collect the datailed data using Premium calls
for venue in dataframe['venue.id']:
    VENUE_ID = venue
    url1 = 'https://api.foursquare.com/v2/venues/{}?oauth_token={}&v={}'.format(VENUE_ID,oauth_token,v)
    resp1 = requests.get(url=url1)
    data1 = json.loads(resp1.text)

    results1 = data1['response']
    dataframe1 = json_normalize(results1)
    dataframe1['venue.categories'] = data1['response']['venue']['categories'][0]['shortName']
    dataframeFull=dataframeFull.append(dataframe1)

#Using the function to transform the APOCH date in to normal date(this function is described in the beggining of this notebook)
dataframeFull['venue.createdAt'] = dataframeFull['venue.createdAt'].apply(epoch_date)
dataframeFull['venue.createdAt'] = pd.to_datetime(dataframeFull['venue.createdAt'])

In [ ]:
#Selecting the columns that is useful to use or to consult
dataframeFull=dataframeFull[['venue.hours.isOpen','venue.categories', 'venue.createdAt', 'venue.id', 'venue.likes.count', 'venue.location.address', 'venue.location.lat', 'venue.location.lng', 'venue.name', 'venue.photos.count', 'venue.price.message', 'venue.price.tier', 'venue.rating', 'venue.ratingSignals', 'venue.shortUrl', 'venue.tips.count']]

#Creating a new Dataframe to be used in the map - Filtering the venues with less than 10 likes.
dfRecent = dataframeFull.loc[(dataframeFull["venue.likes.count"] < 10)]

In [89]:
#Using the folium to create a map with the dataframe filtered and adding labels
map_Restaurant = folium.Map(location=[location.latitude, location.longitude], zoom_start=14)
latitudes = dfRecent['venue.location.lat']
longitudes = dfRecent['venue.location.lng']
labels = "This is the " + dfRecent["venue.name"]
labels = labels + ", here you can find "+ dfRecent['venue.categories'].astype(str) + " food."
labels = labels + ". The current rating is "+ dfRecent['venue.rating'].astype(str) + ", help them to increase, leaving feedbacks"
labels = labels + ' in the Foursquare page: "' + dfRecent['venue.shortUrl'].astype(str) + '"'
labels = labels + ". Consider take-out or delivery from this restaurants. They need our help now more than ever."

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker(
        [lat, lng], 
        popup=folium.Popup(label, parse_html=True)
    ).add_to(map_Restaurant)
    
map_Restaurant